# Import

In [1]:
from keras.models import load_model
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import keras
import yfinance as yf
from matplotlib.backends.backend_pdf import PdfPages
import pickle
import json
from scipy.optimize import minimize
from datetime import datetime, timedelta

# Download stock data

In [2]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # write column headings
        f.write('Date,Open,High,Low,Close,Volume\n')
        
        #for symbol in stock_symbols:
            # load share data for the symbol (Share name) and the given time period
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Auswahl der gewünschten Spalten
        stock_data.to_csv(f, header=False)  # Schreiben der Daten in die Datei

In [3]:
# Enter symbols
stock_symbols = ['ALV.DE', 'DBK.DE', 'VOW3.DE', 'BMW.DE', 'ADS.DE', 'BEI.DE', 'DTE.SG', 'SAP.DE', '1COV.DE', 'BAS.DE', 'EOAN.DE', 'RWE.DE']
start_date = '2014-01-01'
end_date = '2018-12-31'

# load and store the data for every share
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # read CSV and select select the desired columns
    df = pd.read_csv(output_file, usecols=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    
    # Search CSV file for empty lines and remove them
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # filter the empty lines
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the adjusted rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
# store the data frames for different symbols in the dictionary
dfPre = {}

for symbol in stock_symbols:
    # load CSV
    dfPre[symbol] = pd.read_csv(f'stock_data_{symbol}.csv')

    # Drop empty rows
    dfPre[symbol].dropna(inplace=True)

    # transform date to index
    dfPre[symbol].set_index('Date', inplace=True)

    # Check the number of rows
    num_rows = dfPre[symbol].shape[0]

In [5]:
# Check missing values
missing_values_beforePre = {symbol: dfPre[symbol].isnull().values.any() for symbol in stock_symbols}

# missing values are replaced with teh mean of the row before and after the actual row
for symbol in stock_symbols:
    for column in dfPre[symbol].columns:
        missing_valuesPre = dfPre[symbol][column].isnull()
        dfPre[symbol].loc[missing_valuesPre, column] = (dfPre[symbol][column].shift() + dfPre[symbol][column].shift(-1)) / 2

# Check if there are still any missing values 
missing_values_afterPre = {symbol: dfPre[symbol].isnull().values.any() for symbol in stock_symbols}

# Output of the missing values before and after the Treatment
for symbol in stock_symbols:
    print(f"Fehlende Werte vor der Behandlung für {symbol} gefunden:", missing_values_beforePre[symbol])
    print(f"Fehlende Werte nach der Behandlung für {symbol} gefunden:", missing_values_afterPre[symbol])

# Output of the length of the data frame for all symbols
for symbol in stock_symbols:
    print(f"Länge des Datensatzes für {symbol}:", len(dfPre[symbol]))

Fehlende Werte vor der Behandlung für ALV.DE gefunden: False
Fehlende Werte nach der Behandlung für ALV.DE gefunden: False
Fehlende Werte vor der Behandlung für DBK.DE gefunden: False
Fehlende Werte nach der Behandlung für DBK.DE gefunden: False
Fehlende Werte vor der Behandlung für VOW3.DE gefunden: False
Fehlende Werte nach der Behandlung für VOW3.DE gefunden: False
Fehlende Werte vor der Behandlung für BMW.DE gefunden: False
Fehlende Werte nach der Behandlung für BMW.DE gefunden: False
Fehlende Werte vor der Behandlung für ADS.DE gefunden: False
Fehlende Werte nach der Behandlung für ADS.DE gefunden: False
Fehlende Werte vor der Behandlung für BEI.DE gefunden: False
Fehlende Werte nach der Behandlung für BEI.DE gefunden: False
Fehlende Werte vor der Behandlung für DTE.SG gefunden: False
Fehlende Werte nach der Behandlung für DTE.SG gefunden: False
Fehlende Werte vor der Behandlung für SAP.DE gefunden: False
Fehlende Werte nach der Behandlung für SAP.DE gefunden: False
Fehlende Werte

# Read close values at the begin of the time period

In [6]:
# Definition of the time period
# convert in to datetime objekt to substract days for the last day before this period
start_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
end_date_obj = datetime.strptime(end_date, '%Y-%m-%d')
# Substract 5 days for the new start_date and 1 day for the new end_date
# In the next block the last date will be extracted 
new_start_date = start_date_obj - timedelta(days=5)
new_end_date = end_date_obj

print(new_start_date)
print(new_end_date)

# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    download_stock_data(symbol, new_start_date, new_end_date, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned data
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

2018-12-26 00:00:00
2018-12-31 00:00:00



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [7]:
# create data frame for the collected data 
combined_data_begin = pd.DataFrame(columns=['Symbol', 'Close_Value'])

# load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_begin{symbol}.csv'
    # read CSV file 
    df = pd.read_csv(output_file)

    # Select the last row only
    last_row = df.tail(1)

     # Write symbol and close value to the data frame
    data = {'Symbol': symbol, 'Close_Value': last_row['Close'].iloc[0]}
    combined_data_begin = pd.concat([combined_data_begin, pd.DataFrame(data, index=[0])], ignore_index=True)

print(combined_data_begin)

     Symbol  Close_Value
0    ALV.DE   175.139999
1    DBK.DE     6.967000
2   VOW3.DE   138.919998
3    BMW.DE    70.699997
4    ADS.DE   182.399994
5    BEI.DE    91.160004
6    DTE.SG    13.690000
7    SAP.DE    86.930000
8   1COV.DE    43.180000
9    BAS.DE    60.400002
10  EOAN.DE     8.627000
11   RWE.DE    18.965000


# load share value from 2019

In [8]:
start_date = '2019-12-01'
end_date = '2019-12-31'
print(f'Startdatum: {start_date}, Enddatum: {end_date}')
# Load and store the symbols for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2019.csv'
    download_stock_data(symbol, start_date, end_date, output_file)

    # search and delete for empty rows in the CSV 
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed


Startdatum: 2019-12-01, Enddatum: 2019-12-31


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
# list for the las close value of the shares
last_close_values = []

# load for each symbol
for symbol in stock_symbols:
    # file name of the csv data
    csv_file = f'stock_data_{symbol}_2019.csv'
    
    # load CSV data
    df = pd.read_csv(csv_file)
    
    # set date as index
    df.set_index('Date', inplace=True)
    
    # collect just the last close value and write it to the list
    last_close_value = df['Close'].iloc[-1]
    last_close_values.append((symbol, last_close_value))
print(last_close_values[0][1])
# print the list
print("Liste der letzten Close-Werte:")
for symbol, last_close_value in last_close_values:
    print(f"{symbol}: {last_close_value}")

218.3999938964844
Liste der letzten Close-Werte:
ALV.DE: 218.3999938964844
DBK.DE: 6.916999816894531
VOW3.DE: 176.24000549316406
BMW.DE: 73.13999938964844
ADS.DE: 289.79998779296875
BEI.DE: 106.6500015258789
DTE.SG: 13.6899995803833
SAP.DE: 120.31999969482422
1COV.DE: 41.45000076293945
BAS.DE: 67.3499984741211
EOAN.DE: 9.52400016784668
RWE.DE: 27.350000381469727


# Split into blocks of 20 (20 business days per month)

In [10]:
# Dictionary for the splitted data
df_split = {}

# Quantity of the blocks per symbol
block_size = 20

for symbol, df in dfPre.items():
    # Quantity of the rows in the data frame
    num_rows = len(df)

    # Split the data frame in to blocks
    split_dfs = np.array_split(df, num_rows // block_size)

    # Add the solited blocks to the dictionary
    df_split[symbol] = split_dfs

# Output result
for symbol, split_dfs in df_split.items():
    print(f"Symbol: {symbol}")
    for i, df_block in enumerate(split_dfs):
        print(f"Block {i+1}:")
        print(df_block)
        print()

Symbol: ALV.DE
Block 1:
                  Open        High         Low       Close   Volume
Date                                                               
2014-01-02  130.949997  131.399994  127.849998  128.149994  1524683
2014-01-03  128.000000  128.449997  127.150002  127.900002   853713
2014-01-06  127.650002  128.800003  127.599998  128.399994   900878
2014-01-07  128.350006  130.350006  128.149994  130.100006  1189652
2014-01-08  130.149994  130.149994  128.850006  129.399994  1214204
2014-01-09  128.899994  129.899994  127.800003  127.949997  1206159
2014-01-10  128.899994  130.449997  128.850006  129.149994  1098781
2014-01-13  129.350006  129.649994  128.300003  128.500000  1077211
2014-01-14  128.000000  129.399994  127.199997  129.399994  1219362
2014-01-15  131.149994  134.050003  131.100006  133.500000  2233252
2014-01-16  133.800003  134.399994  133.300003  133.550003  1323570
2014-01-17  133.550003  134.350006  132.449997  133.899994  2154664
2014-01-20  132.600006  

                 Open       High        Low      Close    Volume
Date                                                            
2015-02-23  26.105839  26.226328  25.632809  25.918411   8625294
2015-02-24  25.838085  26.217402  25.583721  26.110300   8204639
2015-02-25  26.105839  26.105839  25.735449  25.793461   6183513
2015-02-26  25.829161  26.230789  25.824699  26.038900   7488003
2015-02-27  25.971962  26.217402  25.887175  26.217402   7698562
2015-03-02  26.016588  26.453917  26.016588  26.257565   6781308
2015-03-03  26.248640  26.302191  25.655123  25.681896   7959998
2015-03-04  25.797922  25.971962  25.507858  25.824699   6200420
2015-03-05  25.927338  26.163851  25.699747  26.096912   6538532
2015-03-06  26.114763  26.538704  26.034437  26.315578   6398935
2015-03-09  26.132614  26.520855  26.034437  26.212938   5911466
2015-03-10  26.302191  26.351278  25.722059  26.056749   7350375
2015-03-11  25.927338  26.748444  25.882711  26.721668   9874173
2015-03-12  26.578867  26

                  Open        High         Low       Close   Volume
Date                                                               
2018-01-18  182.080002  182.100006  179.320007  179.960007  1187082
2018-01-19  179.600006  183.720001  179.399994  183.720001  1565822
2018-01-22  182.919998  188.899994  182.279999  188.500000  1764845
2018-01-23  189.660004  192.460007  186.419998  187.259995  1849879
2018-01-24  188.000000  188.979996  184.020004  184.020004  1350565
2018-01-25  182.839996  184.820007  179.460007  181.380005  1492560
2018-01-26  181.820007  182.619995  180.580002  181.899994   834092
2018-01-29  184.899994  185.800003  182.000000  182.080002  1031696
2018-01-30  180.500000  182.020004  178.300003  178.600006  1345896
2018-01-31  178.259995  180.880005  176.660004  177.020004  1292804
2018-02-01  178.520004  181.619995  175.100006  177.600006  1419121
2018-02-02  176.399994  177.100006  171.360001  172.059998  2080247
2018-02-05  170.399994  171.440002  168.699997  

                  Open        High         Low       Close   Volume
Date                                                               
2018-08-07  146.460007  149.100006  146.220001  146.880005   955205
2018-08-08  146.320007  148.720001  145.639999  147.820007   908549
2018-08-09  147.520004  148.800003  146.619995  147.839996   779279
2018-08-10  147.059998  147.339996  143.240005  144.380005  1278027
2018-08-13  143.380005  144.960007  143.300003  143.979996   589321
2018-08-14  144.940002  145.320007  142.339996  142.899994   793502
2018-08-15  142.679993  143.779999  137.460007  138.440002  1962637
2018-08-16  139.919998  140.740005  138.899994  139.440002   995530
2018-08-17  139.440002  139.720001  136.539993  138.740005  1226563
2018-08-20  139.500000  140.619995  138.339996  139.000000   661764
2018-08-21  138.720001  141.160004  137.899994  140.720001  1150636
2018-08-22  139.899994  142.300003  136.660004  138.820007  1585333
2018-08-23  139.020004  139.100006  137.520004  

                  Open        High         Low       Close  Volume
Date                                                              
2017-04-05  177.000000  177.600006  175.500000  175.600006  513493
2017-04-06  174.449997  176.949997  173.949997  176.600006  478895
2017-04-07  176.149994  178.300003  175.500000  178.300003  514932
2017-04-10  178.399994  178.649994  176.649994  177.399994  270234
2017-04-11  176.649994  178.300003  174.649994  175.800003  475036
2017-04-12  176.149994  177.449997  175.250000  177.449997  501507
2017-04-13  177.350006  178.800003  176.750000  178.050003  518083
2017-04-18  178.500000  179.750000  177.949997  177.949997  502509
2017-04-19  177.949997  178.500000  176.649994  177.300003  391724
2017-04-20  177.100006  178.699997  176.550003  178.199997  615875
2017-04-21  178.649994  181.250000  178.149994  180.399994  926045
2017-04-24  186.300003  187.000000  182.899994  184.350006  907294
2017-04-25  185.300003  186.350006  184.300003  185.699997  53

...

                 Open       High        Low      Close  Volume
Date                                                          
2017-05-08  16.650000  16.865000  16.634001  16.806000  119706
2017-05-09  16.781000  16.930000  16.781000  16.823999   73206
2017-05-10  16.816999  16.927999  16.743000  16.816999   99196
2017-05-11  16.813000  16.905001  16.680000  16.763000   90065
2017-05-12  16.754999  17.542999  16.732000  17.474001  276263
2017-05-15  17.533001  17.643999  17.337999  17.462999  119505
2017-05-16  17.450001  17.674000  17.450001  17.645000  149092
2017-05-17  17.648001  17.658001  17.313000  17.316000   89628
2017-05-18  17.388000  17.558001  17.280001  17.495001   82208
2017-05-19  17.518000  17.837999  17.420000  17.785000  102898
2017-05-22  17.868999  18.065001  17.809999  18.010000  138990
2017-05-23  18.059999  18.138000  17.990000  18.013000  155194
2017-05-24  17.950001  18.038000  17.818001  17.858999   68602
2017-05-25  17.882999  17.938999  17.698000  17.830999 

                 Open       High        Low      Close   Volume
Date                                                           
2014-01-02  78.400002  78.720001  76.260002  76.379997  2960768
2014-01-03  76.389999  76.849998  76.029999  76.570000  1707214
2014-01-06  76.209999  76.989998  76.059998  76.419998  1749064
2014-01-07  78.430000  78.860001  78.209999  78.519997  3974187
2014-01-08  77.989998  78.419998  77.400002  78.000000  2642750
2014-01-09  77.720001  78.059998  76.550003  76.720001  3067307
2014-01-10  77.320000  77.980003  77.070000  77.400002  2154328
2014-01-13  77.900002  77.910004  76.989998  77.489998  1784242
2014-01-14  76.669998  77.570000  76.290001  77.570000  2302664
2014-01-15  77.699997  78.510002  77.080002  78.180000  3309532
2014-01-16  78.620003  79.070000  78.070000  78.989998  2794082
2014-01-17  79.169998  80.599998  79.019997  79.489998  4689497
2014-01-20  79.269997  80.089996  78.919998  79.860001  1616046
2014-01-21  80.300003  81.269997  79.739

                 Open       High        Low      Close   Volume
Date                                                           
2017-10-23  90.639999  91.050003  90.129997  91.000000  1532566
2017-10-24  91.000000  91.000000  89.419998  90.110001  2883262
2017-10-25  90.099998  92.129997  90.050003  91.050003  3073321
2017-10-26  91.349998  92.750000  91.169998  92.620003  2698266
2017-10-27  92.800003  94.379997  92.730003  93.830002  3670545
2017-10-30  93.870003  94.059998  93.120003  93.620003  1847535
2017-10-31  93.620003  93.620003  93.620003  93.620003        0
2017-11-01  94.589996  96.290001  94.480003  96.010002  3401739
2017-11-02  95.629997  96.169998  95.410004  95.989998  1454812
2017-11-03  96.169998  97.099998  96.059998  97.010002  2049949
2017-11-06  96.919998  97.480003  96.330002  97.459999  1624704
2017-11-07  97.750000  97.900002  96.330002  96.599998  2093906
2017-11-08  95.849998  96.760002  95.300003  96.669998  1988734
2017-11-09  96.699997  96.989998  94.639

             Open   High    Low  Close    Volume
Date                                            
2018-06-12  8.956  9.088  8.925  9.075  11832489
2018-06-13  9.080  9.100  8.916  8.971  13439009
2018-06-14  8.912  9.313  8.890  9.272  17944890
2018-06-15  9.329  9.524  9.232  9.489  35695648
2018-06-18  9.459  9.470  9.242  9.295  13099116
2018-06-19  9.211  9.262  9.120  9.245  11640968
2018-06-20  9.255  9.335  9.122  9.201  10728039
2018-06-21  9.218  9.266  9.086  9.097  10266962
2018-06-22  9.087  9.099  8.998  9.096   8679329
2018-06-25  9.050  9.102  8.993  9.010   9066667
2018-06-26  9.045  9.241  9.026  9.163  10949376
2018-06-27  9.165  9.229  9.055  9.195   9228346
2018-06-28  9.180  9.248  9.055  9.093  10181875
2018-06-29  9.143  9.217  9.106  9.154  10563743
2018-07-02  9.020  9.268  9.015  9.253  12282037
2018-07-03  9.290  9.524  9.262  9.479  14259061
2018-07-04  9.478  9.700  9.475  9.700  13988937
2018-07-05  9.664  9.834  9.664  9.814  15351143
2018-07-06  9.840  9

              Open    High     Low   Close   Volume
Date                                               
2016-08-22  14.470  14.775  14.420  14.605  2874688
2016-08-23  14.680  14.730  14.500  14.605  1951746
2016-08-24  14.550  14.770  14.350  14.675  1585592
2016-08-25  14.690  14.720  14.520  14.645  1803072
2016-08-26  14.630  14.905  14.550  14.785  1690859
2016-08-29  14.675  14.770  14.515  14.710  1216308
2016-08-30  14.770  14.940  14.685  14.750  1777823
2016-08-31  14.710  14.765  14.540  14.665  2673967
2016-09-01  14.710  14.830  14.165  14.230  2932347
2016-09-02  14.310  15.095  14.300  15.030  4178942
2016-09-05  14.955  15.295  14.910  15.220  2871648
2016-09-06  15.220  15.350  15.105  15.155  2083260
2016-09-07  15.100  15.240  14.770  15.195  3327605
2016-09-08  15.160  15.310  14.965  15.180  3054376
2016-09-09  15.095  15.250  14.835  14.900  3305087
2016-09-12  14.505  14.645  14.275  14.645  4212905
2016-09-13  14.700  14.790  14.420  14.505  3499264
2016-09-14  

# Calculate the return for every month

In [11]:
# Dictionary for the increase/decrease per symbol
change_per_symbol = {}

for symbol, split_dfs in df_split.items():
    changes = []

    # Iterate over every block of the current symbol
    for df_block in split_dfs:
        # Extract the first and the lat close value of the current block
        first_close = df_block.iloc[0]['Close']
        last_close = df_block.iloc[-1]['Close']

        # Calculate the change of the close value
        change = (last_close - first_close) / first_close * 100  # in Prozent
        changes.append(change)

    # Save the Change for the current symbol
    change_per_symbol[symbol] = changes

# Output result
for symbol, changes in change_per_symbol.items():
    print(f"Symbol: {symbol}")
    for i, change in enumerate(changes):
        print(f"Block {i+1}: {change:.2f}%")
    print()


Symbol: ALV.DE
Block 1: -2.50%
Block 2: 4.77%
Block 3: -2.19%
Block 4: 0.61%
Block 5: 0.40%
Block 6: -1.33%
Block 7: 3.71%
Block 8: 3.68%
Block 9: 5.28%
Block 10: -4.02%
Block 11: 11.12%
Block 12: 1.43%
Block 13: 6.07%
Block 14: -1.41%
Block 15: 9.36%
Block 16: 2.00%
Block 17: -6.48%
Block 18: -6.92%
Block 19: 10.28%
Block 20: -1.04%
Block 21: -6.85%
Block 22: 3.09%
Block 23: 8.71%
Block 24: 3.34%
Block 25: -3.60%
Block 26: -8.34%
Block 27: -0.71%
Block 28: 0.49%
Block 29: 8.52%
Block 30: -1.01%
Block 31: -13.74%
Block 32: 4.74%
Block 33: 2.46%
Block 34: 0.58%
Block 35: 5.34%
Block 36: 9.45%
Block 37: 1.47%
Block 38: 2.30%
Block 39: -2.47%
Block 40: 8.44%
Block 41: 2.91%
Block 42: 1.05%
Block 43: 0.26%
Block 44: -0.55%
Block 45: 2.70%
Block 46: 1.22%
Block 47: 2.95%
Block 48: 6.09%
Block 49: 0.43%
Block 50: 0.33%
Block 51: 0.72%
Block 52: -7.61%
Block 53: -2.32%
Block 54: 0.45%
Block 55: 0.51%
Block 56: -5.87%
Block 57: -0.10%
Block 58: 3.47%
Block 59: -2.23%
Block 60: 6.23%
Block 61: 

# Calculate the mean return over the period of 4 years

In [12]:
# Dictionary for the mean return per symbol
average_return_per_symbol = {}

for symbol, changes in change_per_symbol.items():
    # Calculate the mean return for the current symbol
    average_return = sum(changes) / len(changes)
    average_return_per_symbol[symbol] = average_return

# Output result
for symbol, avg_return in average_return_per_symbol.items():
    print(f"Durchschnittliche Rendite für {symbol}: {avg_return:.2f}%")
    
# Convert mean return to DataFrame
average_return_per_symbol_df = pd.DataFrame(list(average_return_per_symbol.items()), columns=['Symbol', 'Average_Return'])
print(average_return_per_symbol_df)


Durchschnittliche Rendite für ALV.DE: 0.88%
Durchschnittliche Rendite für DBK.DE: -1.60%
Durchschnittliche Rendite für VOW3.DE: -0.06%
Durchschnittliche Rendite für BMW.DE: 0.03%
Durchschnittliche Rendite für ADS.DE: 1.66%
Durchschnittliche Rendite für BEI.DE: 0.60%
Durchschnittliche Rendite für DTE.SG: 0.52%
Durchschnittliche Rendite für SAP.DE: 0.81%
Durchschnittliche Rendite für 1COV.DE: 1.52%
Durchschnittliche Rendite für BAS.DE: -0.07%
Durchschnittliche Rendite für EOAN.DE: -0.03%
Durchschnittliche Rendite für RWE.DE: -0.02%
     Symbol  Average_Return
0    ALV.DE        0.883068
1    DBK.DE       -1.600692
2   VOW3.DE       -0.063100
3    BMW.DE        0.034505
4    ADS.DE        1.661904
5    BEI.DE        0.600900
6    DTE.SG        0.515062
7    SAP.DE        0.811723
8   1COV.DE        1.517093
9    BAS.DE       -0.066519
10  EOAN.DE       -0.027756
11   RWE.DE       -0.024340


In [13]:
# Dictionary for the standard deviation per symbol
standard_deviation_per_symbol = {}

for symbol, changes in change_per_symbol.items():
    
    # Calculate the difference to the power of 2
    squared_diff = [(change - average_return_per_symbol[symbol]) ** 2 for change in changes]
    
    # Sum up the differences
    sum_squared_diff = sum(squared_diff)

    # Calculate the standard deviation
    standard_deviation = np.sqrt(sum_squared_diff / (len(changes) - 1))
    
    # Save the standard deviation in the dictionary
    standard_deviation_per_symbol[symbol] = standard_deviation

# Output result
for symbol, std_dev in standard_deviation_per_symbol.items():
    print(f"Standardabweichung für {symbol}: {std_dev:.2f}")
    
# Convert mean return to DataFrame
standard_deviation_per_symbol_df = pd.DataFrame(list(standard_deviation_per_symbol.items()), columns=['Symbol', 'Standard_Deviation'])
print(standard_deviation_per_symbol_df)

Standardabweichung für ALV.DE: 4.99
Standardabweichung für DBK.DE: 9.28
Standardabweichung für VOW3.DE: 9.74
Standardabweichung für BMW.DE: 6.06
Standardabweichung für ADS.DE: 7.44
Standardabweichung für BEI.DE: 4.76
Standardabweichung für DTE.SG: 5.18
Standardabweichung für SAP.DE: 5.99
Standardabweichung für 1COV.DE: 8.70
Standardabweichung für BAS.DE: 5.91
Standardabweichung für EOAN.DE: 7.33
Standardabweichung für RWE.DE: 10.35
     Symbol  Standard_Deviation
0    ALV.DE            4.992595
1    DBK.DE            9.279483
2   VOW3.DE            9.741308
3    BMW.DE            6.059151
4    ADS.DE            7.442931
5    BEI.DE            4.762488
6    DTE.SG            5.182544
7    SAP.DE            5.988272
8   1COV.DE            8.702709
9    BAS.DE            5.911711
10  EOAN.DE            7.326331
11   RWE.DE           10.354041


# Historic data from 2018 for the covarianz matrix

In [14]:
# Definition of the time period
start_date2018 = '2018-01-01'
end_date2018 = '2018-12-31'
print(end_date2018)
# Load and store the data for every symbol (share)
for symbol in stock_symbols:
    output_file = f'stock_data_{symbol}_2018.csv'
    download_stock_data(symbol, start_date2018, end_date2018, output_file)

    # Search and delete empty rows in the CSV file
    with open(output_file, 'r') as file:
        lines = file.readlines()

    # Filter the empty rows
    lines = [line.strip() for line in lines if line.strip()]

    # Overwrite the file with the cleaned rows
    with open(output_file, 'w') as file:
        file.write('\n'.join(lines))


[*********************100%%**********************]  1 of 1 completed

2018-12-31



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# Combine historical data to one file

In [15]:
# Data frame for the combined data
combined_historical2018_data = pd.DataFrame()

# load and combine the data for each share
for symbol in stock_symbols:
    # file name for the csv file
    csv_file = f'stock_data_{symbol}_2018.csv'
    
    # read CSV file and set date to index
    df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
    
    # change name of the close column
    df.rename(columns={'Close': symbol}, inplace=True)
    
    # write data to the combined file
    combined_historical2018_data = pd.concat([combined_historical2018_data, df[symbol]], axis=1)

# drop rows with NaN values
combined_historical2018_data.dropna(inplace=True)
    
# create a CSV file for the data
combined_historical2018_data.to_csv('combined_historical2018_stock_data.csv')

print(combined_historical2018_data)

                ALV.DE     DBK.DE     VOW3.DE     BMW.DE      ADS.DE  \
2018-01-02  192.699997  15.958000  165.699997  86.400002  167.149994   
2018-01-03  193.000000  15.910000  171.440002  86.860001  168.050003   
2018-01-04  195.500000  16.332001  174.440002  87.480003  170.250000   
2018-01-05  198.559998  15.490000  179.199997  88.500000  172.050003   
2018-01-08  198.860001  15.340000  179.839996  89.669998  172.750000   
...                ...        ...         ...        ...         ...   
2018-12-19  175.940002   7.530000  146.880005  73.449997  187.050003   
2018-12-20  174.960007   7.000000  143.899994  71.839996  183.750000   
2018-12-21  175.020004   7.042000  143.300003  71.930000  184.750000   
2018-12-27  172.160004   6.750000  137.440002  69.860001  180.100006   
2018-12-28  175.139999   6.967000  138.919998  70.699997  182.399994   

                BEI.DE  DTE.SG     SAP.DE    1COV.DE     BAS.DE  EOAN.DE  \
2018-01-02   96.199997  14.872  92.800003  85.639999  91.30

# Mean-Variance Optimization

In [16]:
def mean_variance_optimization(expected_returns, standard_deviations, covariance_matrix):
    n = len(expected_returns)
    initial_weights = np.array([1/n] * n)  # Start value of the weighs
    bounds = [(0, 1)] * n  # Border of the weighs (0-100% for every symbol)

    # Minimize the negative sharpe ratio
    def negative_sharpe(weights, expected_returns, standard_deviations, covariance_matrix):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
        return -portfolio_return / portfolio_volatility

    result = minimize(negative_sharpe, initial_weights, args=(expected_returns, standard_deviations, covariance_matrix), bounds=bounds, constraints={'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    return result.x

In [17]:
def allocate_portfolio(historical_data, initial_capital, expected_returns_df, standard_deviations_df):
    # Calculate expected returns based on the predictions
    expected_returns = expected_returns_df['Average_Return'].values
    
    # Calculate standard deviations based on the predictions
    standard_deviations = standard_deviations_df['Standard_Deviation'].values
    
    # Calculate the covariance matrix of the returns
    covariance_matrix = historical_data.cov()
           
    print(covariance_matrix)
    
    # Perform mean-variance optimization to obtain optimal weights
    optimal_weights = mean_variance_optimization(expected_returns, standard_deviations, covariance_matrix)

    # Calculate the allocation of assets based on the optimal weights and the available capital
    asset_allocation = initial_capital * optimal_weights

    return asset_allocation

In [18]:
initial_capital = 1000 # 1000€ start capital
portfolio_allocation = allocate_portfolio(combined_historical2018_data, initial_capital, average_return_per_symbol_df, standard_deviation_per_symbol_df)
print(portfolio_allocation)

            ALV.DE     DBK.DE     VOW3.DE     BMW.DE      ADS.DE     BEI.DE  \
ALV.DE   55.719135  11.832723   69.633719  29.543884  -14.913587   0.700969   
DBK.DE   11.832723   4.459489   21.177797  10.619505  -14.403096  -0.613004   
VOW3.DE  69.633719  21.177797  173.761508  68.923353  -60.940672  -4.044547   
BMW.DE   29.543884  10.619505   68.923353  39.954812  -18.970997   3.151594   
ADS.DE  -14.913587 -14.403096  -60.940672 -18.970997  151.098444   7.531624   
BEI.DE    0.700969  -0.613004   -4.044547   3.151594    7.531624  13.783059   
DTE.SG    1.824277   0.309639    1.939910   0.824193   -0.172976   0.808876   
SAP.DE   -7.402701  -4.671514  -35.207519  -7.149319   32.796906  19.146100   
1COV.DE  52.288467  23.954566  105.094400  71.570572  -67.992106   6.309557   
BAS.DE   38.744048  16.118842   87.074221  52.709029  -45.541645   8.408377   
EOAN.DE  -0.464208  -0.189888   -0.824805   0.054390    0.739193   0.973982   
RWE.DE   -3.476472  -1.526798   -8.780525  -0.837524

In [19]:
# Initialisation of the portfolio
portfolio_value = 0
# initialize the list for the quantity of the shares, which have to be purchased
shares_to_buy_list = []
# Purchase the shares based on the allocations
for i, allocation in enumerate(portfolio_allocation):
    # Calculate the quantity of the shares, which should be purchased with this allocation
    shares_to_buy = (allocation) / combined_data_begin['Close_Value'][i]
    shares_to_buy_list.append(shares_to_buy)
    print(shares_to_buy)

    # Calculate the value of the stock of the purchased shares
    value_of_stock = shares_to_buy * combined_data_begin['Close_Value'][i]
    print(value_of_stock)
    
    # Add the value of the purchased share to the portfolio
    portfolio_value += value_of_stock
    print(portfolio_value)
    print("-----------------------------")

# Output of the total value of the portfolio at the end of the period to be predicted
print("Gesamtwert des Portfolios am Ende des ersten Monats von 2019:", portfolio_value)
print(shares_to_buy_list)

2.466068560969556e-11
4.3190724626303924e-09
4.3190724626303924e-09
-----------------------------
4.990330790427981e-11
3.4767634654984947e-10
4.666748809180242e-09
-----------------------------
5.5016120395523385e-12
7.642839344608584e-10
5.431032743641101e-09
-----------------------------
2.2055788770046558e-12
1.559344198733366e-10
5.586967163514437e-09
-----------------------------
0.03347952016526722
6.106664273801968
6.106664279388935
-----------------------------
1.731645171070699e-11
1.5785678013627891e-09
6.106664280967503
-----------------------------
72.47529191573365
992.1867159145509
998.2933801955184
-----------------------------
0.0
0.0
998.2933801955184
-----------------------------
0.039523385667579944
1.706619805187682
1000.000000000706
-----------------------------
4.350017585615578e-12
2.62741068808781e-10
1000.0000000009687
-----------------------------
0.0
0.0
1000.0000000009687
-----------------------------
0.0
0.0
1000.0000000009687
-----------------------------

In [20]:
# Define portfolio value
new_portfolio_value = 0
# Loop over the indices of the two lists
for i in range(len(shares_to_buy_list)):
    # convert to float
    last_close_value = float(last_close_values[i][1])
    print(last_close_value)
    print(shares_to_buy_list[i])
    #Calculate the new value of the share and add the value to the list
    new_portfolio_value += (last_close_value * shares_to_buy_list[i])
    print(new_portfolio_value)
    print("----------------------------")

print("Wert des Portfolios:")
print(new_portfolio_value)

218.3999938964844
2.466068560969556e-11
5.3858935866406305e-09
----------------------------
6.916999816894531
4.990330790427981e-11
5.731074758276965e-09
----------------------------
176.24000549316406
5.5016120395523385e-12
6.700678894348927e-09
----------------------------
73.13999938964844
2.2055788770046558e-12
6.861994932066869e-09
----------------------------
289.79998779296875
0.03347952016526722
9.702364542070887
----------------------------
106.6500015258789
1.731645171070699e-11
9.702364543917687
----------------------------
13.6899995803833
72.47529191573365
1001.8890804584686
----------------------------
120.31999969482422
0.0
1001.8890804584686
----------------------------
41.45000076293945
0.039523385667579944
1003.5273248245437
----------------------------
67.3499984741211
4.350017585615578e-12
1003.5273248248367
----------------------------
9.52400016784668
0.0
1003.5273248248367
----------------------------
27.350000381469727
0.0
1003.5273248248367
--------------------